In [ ]:
#| default_exp config

In [ ]:
#| export
from __future__ import annotations

# config

> Configuration loading and validation for ctxt


<!-- # Prologue -->


In [ ]:
#| export
from importlib.resources import files
from pathlib import Path

import yaml


In [ ]:
#| export
from pote.common import val_at
from pote.display import RenderJSON


In [ ]:
#| hide
from fastcore.test import *
from nbdev.showdoc import *


# Default config

In [ ]:
#| exporti

_DEFAULT_CONFIG = {
    'templates': {
        'meta': {
            'path': ["meta/**/*.md", "meta/**/*.yml"]
        },
        'Cursor': {
            'path': [".cursor/rules/"]
        }
    },
    'tags': {
        'Project-Name': 'myproject',
        'User-Alias': 'user',
    },
    'general': {
        'ignore': [],
        'warn_missing_tags': True,
        'strict_mode': False,
        'max_recursion': 10,
        'preserve_timestamps': False
    }
}

def _load_default_config() -> dict:
    "Load default config from package resources"
    try:
        default_yml = files('ctxt').joinpath('static/default.ctxt.yml')
        with default_yml.open('r') as f: return yaml.safe_load(f)
    except Exception as e: return _DEFAULT_CONFIG

_DEFAULT_CONFIG = _load_default_config()

# Configuration Loading

In [ ]:
#| exporti
CONFIG_FILENAME = '.ctxt.yml'

In [ ]:
#| export
def find_config(start_path: str | Path = '.') -> Path | None:
    "Search up directory tree for .ctxt.yml config file"
    current = Path(start_path).resolve()
    for parent in [current, *current.parents]:
        config_path = parent / CONFIG_FILENAME
        if config_path.exists(): return config_path
    return None

In [ ]:
config_path = find_config()
assert config_path is not None  # silly Pylance doesn't understand fastcore test_eq, so use assert here to avoid type warning (Hate Wiggly Reds)
test_eq(config_path.name, CONFIG_FILENAME)

In [ ]:
#| export
def load_config(path: str | Path = '.') -> dict:
    "Load and parse `CONFIG_FILENAME` config file, use defaults if not found"
    config, config_path = None, find_config(path)
    if config_path is not None: 
        with open(config_path, 'r') as f: config = yaml.safe_load(f)
    merged = _DEFAULT_CONFIG.copy()
    if config is not None:
        for key in config:
            if isinstance(config[key], dict) and key in merged:
                merged[key] = {**merged[key], **config[key]}
            else:
                merged[key] = config[key]
    return merged

In [ ]:
config = load_config()
test_is(config is not None, True)
test_is('tags' in config, True)

RenderJSON(config, init_level=2, max_height=400).display()

# Configuration Validation


In [ ]:
#| export
def validate_config(config: dict) -> tuple[bool, list[str]]:
    "Validate config structure, return (is_valid, errors)"
    errors = []
    if 'tags' in config and not isinstance(config['tags'], dict):
        errors.append("'tags' section must be a dictionary")
    if 'templates' in config and not isinstance(config['templates'], dict):
        errors.append("'templates' section must be a dictionary")
    if 'general' in config and not isinstance(config['general'], dict):
        errors.append("'general' section must be a dictionary")
    return (len(errors) == 0, errors)

In [ ]:
is_valid, errors = validate_config(config)
test_eq(is_valid, True)
test_eq(errors, [])

In [ ]:
invalid_config = {'tags': 'not-a-dict', 'general': []}
is_valid, errors = validate_config(invalid_config)
test_eq(is_valid, False)
test_eq(len(errors), 2)

# Tag Retrieval


In [ ]:
#| export
def get_tag(config: dict, tag_name: str, default: str | None = None) -> str:
    "Get tag value from config, with optional default"
    return val_at(config, f'tags.{tag_name}', default)  # type: ignore

In [ ]:
project_name = get_tag(config, 'Project-Name')
test_eq(project_name, 'ctxt')

missing = get_tag(config, 'NonExistent', 'default_value')
test_eq(missing, 'default_value')

user_alias = get_tag(config, 'User-Alias')
test_eq(user_alias, 'vic')

In [ ]:
#| export
def get_all_tags(config: dict) -> dict[str, str]:
    "Get all tags from config as a flat dictionary"
    return config.get('tags', {})

In [ ]:
all_tags = get_all_tags(config)
test_is(isinstance(all_tags, dict), True)
test_is('Project-Name' in all_tags, True)
test_is('User-Alias' in all_tags, True)

In [ ]:
# Test template config access
template_cfg = config['templates']
test_is(isinstance(template_cfg, dict), True)

vendors = config['templates']
test_is(isinstance(vendors, dict), True)

meta_cfg = config['templates']['meta']
test_is(isinstance(meta_cfg, dict), True)

options = config['general']
test_is(isinstance(options, dict), True)
test_eq(options['max_recursion'], 10)  # default value

----
<!-- # Colophon -->

In [ ]:
#|hide
#|eval: false
import shutil

import fastcore.all as FC
import nbdev
from nbdev.clean import nbdev_clean

In [ ]:
#|hide
#|eval: false

if FC.IN_NOTEBOOK:
    static_src = Path('static')
    static_dst = Path('../ctxt/static')
    if static_src.exists():
        static_dst.mkdir(exist_ok=True)
        for item in static_src.iterdir():
            if item.is_file():
                shutil.copy2(item, static_dst / item.name)
    
    nb_path = '01_config.ipynb'
    # nbdev_clean(nb_path)
    nbdev.nbdev_export(nb_path)